In [1]:
import matplotlib.pyplot as plt
import hyperspy.api as hs
import numpy as np
import exspy
import scipy
import os
import mplcyberpunk
import matplotlib 
import h5py
import hdf5plugin
from py4Dutils.Added_Functions_For_EELS_Mapping import Set_Wrong_Pixels_to_NaN, Set_Wrong_Pixels_to_Zero, Set_Wrong_Pixels_to_MeanValue
plt.style.use('cyberpunk')
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 
#need to be set for the interactive GUI
%matplotlib ipympl

In [2]:
## test readout h5 file from recorder
filepath = "/masc_shared/ag_volz/Franzi/2024_07_16_Li_Arg_STeel/eels"
filename_save = 'Arg_li_eels_OM'
# filename = 'eels_2c_300umev_001'  #EELS Dataset
# filename = 'NBD_08mrad_2c_4cm_001'  #NBD Dataset

EELS_File= h5py.File(os.path.join(filepath,'Arg_li_eels_data_000001.h5'),'r') 

In [36]:
EELS_MAP = EELS_File['entry/data/data']
EELS_MAP = hs.signals.Signal2D(EELS_MAP)
EELS_MAP.set_signal_type('EELS')

In [13]:
EELS_MAP.axes_manager

Navigation axis name,size,index,offset,scale,units
,17415,0,0.0,1.0,
Signal axis name,size,,offset,scale,units
,512,,0.0,1.0,
,32,,0.0,1.0,


In [14]:
EELS_MAP.plot()

In [37]:
## crop to correct size
x_scan_dim = 129 #gatan #cols
y_scan_dim = 128 #gatan #rows
expected_length = x_scan_dim*y_scan_dim

chosen_length = 16605
offset = chosen_length-expected_length
additional_lines = np.floor(offset/x_scan_dim).astype(int)

start_value = 366
end_value = start_value+expected_length+additional_lines*x_scan_dim

#crop the dataset
EELS_MAP.crop(0, start=start_value, end=end_value)


In [38]:
## reshape and retile the dataset
test = EELS_MAP.data.reshape(y_scan_dim,x_scan_dim+additional_lines,32,512)
test = np.tile(test,(1,2,1,1)) #tiling

## convert array to EELS MAP to visualize the reshaping and tiling
EELS_MAP_reshaped =  hs.signals.Signal2D(test)
EELS_MAP_reshaped.set_signal_type('EELS')
EELS_MAP_reshaped.plot()

In [39]:
## induce row shift by indexing
slope = 1.75 ## adjust slope to match the row shift
roll_list = np.ceil(np.arange(1,x_scan_dim+additional_lines,1)/slope).astype(int)*-1 ## created shift list
print('Shift list', roll_list)

Shift list [ -1  -2  -2  -3  -3  -4  -4  -5  -6  -6  -7  -7  -8  -8  -9 -10 -10 -11
 -11 -12 -12 -13 -14 -14 -15 -15 -16 -16 -17 -18 -18 -19 -19 -20 -20 -21
 -22 -22 -23 -23 -24 -24 -25 -26 -26 -27 -27 -28 -28 -29 -30 -30 -31 -31
 -32 -32 -33 -34 -34 -35 -35 -36 -36 -37 -38 -38 -39 -39 -40 -40 -41 -42
 -42 -43 -43 -44 -44 -45 -46 -46 -47 -47 -48 -48 -49 -50 -50 -51 -51 -52
 -52 -53 -54 -54 -55 -55 -56 -56 -57 -58 -58 -59 -59 -60 -60 -61 -62 -62
 -63 -63 -64 -64 -65 -66 -66 -67 -67 -68 -68 -69 -70 -70 -71 -71 -72 -72
 -73 -74]


In [40]:
## shift by indexing
rows, column_indices = np.ogrid[:test.shape[0], :test.shape[1]]
column_indices
# Always use a negative shift, so that column_indices are valid.
# Alternative: r %= A.shape[1]
roll_list[roll_list < 0] += test.shape[1]
column_indices = column_indices - roll_list[:, np.newaxis]
roll_list.shape
result = test[rows, column_indices,:,:]

In [41]:
## convert array to EELS MAP and crop
EELS_MAP_reshaped =  hs.signals.Signal2D(result)
EELS_MAP_reshaped.set_signal_type('EELS')
## make final crop
EELS_MAP_reshaped.crop(0,start = 21, end = x_scan_dim+21)
EELS_MAP_reshaped.plot()

In [42]:
## store the converted dataset
EELS_MAP_reshaped.save(filename=os.path.join(filepath,filename_save), overwrite=True, extension='hspy')